In [ ]:
import pandas as pd
from emutools.tex import StandardTexDoc
import arviz as az
from inputs.constants import ANALYSIS_START_DATE, ANALYSIS_END_DATE, SUPPLEMENT_PATH
from aust_covid.plotting import plot_key_outputs, plot_cdr_examples, plot_subvariant_props, plot_dispersion_examples
from aust_covid.calibration import get_priors, get_targets
from emutools.calibration import get_sampled_outputs
import plotly.graph_objects as go
from emutools.calibration import round_sigfig

In [ ]:
import plotly.express as px

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')

In [ ]:
spaghetti = pd.read_csv('spaghetti.csv', header=[0, 1, 2], index_col=[0])
burnt_idata = az.from_netcdf('idata_for_spaghetti.nc')
targets = get_targets(app_doc)

In [ ]:
fig = go.Figure()
indicator_spaghetti = spaghetti['adult_seropos_prop']
data = pd.DataFrame()
for col in indicator_spaghetti.columns:
    chain, draw = col
    data['values'] = indicator_spaghetti[col]
    variables = burnt_idata.posterior.sel(chain=int(chain), draw=int(draw)).variables
    info = {i: float(j) for i, j in dict(burnt_idata.posterior.sel(chain=int(chain), draw=int(draw)).variables).items()}
    for param in info:
        data[param] = int(info[param]) if param in ['chain', 'draw'] else round_sigfig(info[param], 3)
    lines = px.line(data, y='values', hover_data=data.columns, marker=marker)
    fig.add_traces(lines.data)
fig